## Antenna traffic single-step evaluation (Chronos)
We evaluate Chronos models on histo_trafic_....csv files by forecasting the last time step for each antenna sector using all prior measurements as context.
The helper scripts `dataprocessing.py` and `single_eval.py` handle preprocessing and RMSE scoring. The cells below run on device; edit `MODEL_IDS` to try more Chronos variants later and 'DATASET' to select the right dataset.jsonl to work on.


In [ ]:
import pandas as pd

from scripts.notebook_helpers import (
    RESULTS_DIR,
    device,
    ensure_processed_data,
    evaluate_model,
)

DATASET = "instant"  # choose among: "orignal", "instant", "instant_short"
print("Using device:", device)
print("Using dataset:", DATASET)

ensure_processed_data(DATASET)



In [ ]:
MODEL_IDS = [
    "amazon/chronos-2",
    "amazon/chronos-bolt-tiny",
    "amazon/chronos-bolt-mini",
    "amazon/chronos-bolt-small",
    "amazon/chronos-bolt-base",
    "amazon/chronos-t5-tiny",
    "amazon/chronos-t5-mini",
    "amazon/chronos-t5-small",
    "amazon/chronos-t5-base",
    "amazon/chronos-t5-large",
]
NUM_SAMPLES = 32  # tweak for slower/fast CPU evaluations


In [ ]:
results = [evaluate_model(mid, num_samples=NUM_SAMPLES, dataset=DATASET) for mid in MODEL_IDS]
summary_df = pd.DataFrame(
    {
        "model_id": [r["model_id"] for r in results],
        "rmse": [r["rmse"] for r in results],
        "num_series": [r["num_series"] for r in results],
        "num_samples": [r["num_samples"] for r in results],
    }
)
summary_df


In [ ]:
PARAM_COUNTS = {
    "amazon/chronos-2": "120M",
    "amazon/chronos-bolt-tiny": "9M",
    "amazon/chronos-bolt-mini": "21M",
    "amazon/chronos-bolt-small": "48M",
    "amazon/chronos-bolt-base": "205M",
    "amazon/chronos-t5-tiny": "8M",
    "amazon/chronos-t5-mini": "20M",
    "amazon/chronos-t5-small": "46M",
    "amazon/chronos-t5-base": "200M",
    "amazon/chronos-t5-large": "710M",
}

order = MODEL_IDS
summary_ranked = summary_df.copy()
summary_ranked["parameters"] = summary_ranked["model_id"].map(PARAM_COUNTS)
summary_ranked["model_id"] = pd.Categorical(summary_ranked["model_id"], categories=order, ordered=True)
summary_ranked = summary_ranked.sort_values("model_id")[["model_id", "rmse", "parameters"]]

# Save the ranked summary to a CSV file
summary_ranked.to_csv(RESULTS_DIR / f"model_performance_summary_{DATASET}.csv", index=False)
summary_ranked